# Port Mirroring

This notebook does following: 
- Slice1: Creates a slice with an isolated local Ethernet and connect compute nodes to it
- Slice2: Creates a slice with a compute node which mirrors the traffic from Slice1.


## Import the FABlib Library


In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

try:
    fablib = fablib_manager()
                     
    fablib.show_config()
except Exception as e:
    print(f"Exception: {e}")

In [ ]:
from plugins import Plugins
import traceback
try:
    Plugins.load()
except Exception as e:
    traceback.print_exc()

## Create the Experiment Slice - Slice1

The following creates two nodes with basic NICs connected to an isolated WAN Ethernet.  

Two nodes are created and one NIC component is added to each node. They are then connected via `l2network`. 

NIC component models options:
- NIC_Basic: 100 Gbps Mellanox ConnectX-6 SR-IOV VF (1 Port)
- NIC_ConnectX_5: 25 Gbps Dedicated Mellanox ConnectX-5 PCI Device (2 Ports) 
- NIC_ConnectX_6: 100 Gbps Dedicated Mellanox ConnectX-6 PCI Device (2 Ports) 

In [ ]:
slice_name_1 = 'MySlice'
slice_name_2 = 'MySlice-port-mirror'
#[site1,site2, site3]  = fablib.get_random_sites(count=3)
site1='UTAH'
site2='DALL'
site3='UTAH'
print(f"Sites: {site1}, {site2}, {site3}")

node1_name = 'Node1'
node2_name = 'Node2'
node3_name = 'Node3'
network_name_1='net1'
network_name_2='port-mirror'
node1_nic_name = 'nic1'
node2_nic_name = 'nic2'
node3_nic_name = 'nic3'
model = 'NIC_ConnectX_5'

In [ ]:
try:
    #Create Slice
    slice1 = fablib.new_slice(name=slice_name_1)

    # Node1
    node1 = slice1.add_node(name=node1_name, site=site1)
    iface1 = node1.add_component(model=model, name=node1_nic_name).get_interfaces()[0]
    
    # Node2
    node2 = slice1.add_node(name=node2_name, site=site2)
    iface2 = node2.add_component(model=model, name=node2_nic_name).get_interfaces()[0]
    
    # Network
    net1 = slice1.add_l2network(name=network_name_1, interfaces=[iface1, iface2])

    #Submit Slice Request
    slice1.submit()
except Exception as e:
    print(f"Exception: {e}")

## List attributes of Slice1

In [ ]:
slice1 = fablib.get_slice(name=slice_name_1)
slice1.show()
slice1.list_nodes()
slice1.list_networks()
slice1.list_interfaces()

## Determine the port names of Slice1 to be used in Slice2 (Port Mirroring Slice)

In [ ]:
slice1 = fablib.get_slice(name=slice_name_1)
network = slice1.get_network(name=network_name_1)
fim_ns = network.get_fim_network_service()

for ifs in fim_ns.interfaces.values():
    if node1_nic_name in ifs.name:
        iface1_local_name = ifs.labels.local_name
        iface1_device_name = ifs.labels.device_name
    elif node2_nic_name in ifs.name:
        iface2_local_name = ifs.labels.local_name
        iface2_device_name = ifs.labels.device_name

print(f"{node1_name} nic: {node1_nic_name}, local_name: {iface1_local_name}, device_name: {iface1_device_name}")
print(f"{node2_name} nic: {node2_nic_name}, local_name: {iface2_local_name}, device_name: {iface2_device_name}")


node_1_port_name=f'port+{iface1_device_name}:{iface1_local_name}'
node_2_port_name=f'port+{iface2_device_name}:{iface2_local_name}'

print(f"Node 1 port name: {node_1_port_name}")
print(f"Node 2 port name: {node_2_port_name}")


## Create Slice2 with port mirroring

In [ ]:
try:
    #Create Slice
    slice2 = fablib.new_slice(name=slice_name_2)
    # Node3
    node3 = slice2.add_node(name=node3_name, site=site3)
    iface3 = node3.add_component(model=model, name=node3_nic_name).get_interfaces()[0]
        
    slice2.get_fim_topology().add_port_mirror_service(name=network_name_2, from_interface_name=node_1_port_name,
                                                      to_interface=iface3.get_fim_interface())

    #Submit Slice Request
    slice2.submit()
except Exception as e:
    print(f"Exception: {e}")

## List Attributes of Slice2(Port Mirror Slice)

In [ ]:
slice2 = fablib.get_slice(name=slice_name_2)
slice2.show()
slice2.list_nodes()
slice2.list_networks()
slice2.list_interfaces()

## Configure IP Addresses on Slice1

### Pick a Subnet

Create subnet and list of available IP addresses. All object are Python IP managment objects. You can use either IPv4 or IPv6 subents and addresses.

In [ ]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

try:
    subnet = IPv4Network("192.168.1.0/24")
    available_ips = list(subnet)[1:]
except Exception as e:
    print(f"Exception: {e}")

### Configure Node1

In [ ]:
try:
    node1 = slice1.get_node(name=node1_name)        
    node1_iface = node1.get_interface(network_name=network_name_1) 
    node1_addr = available_ips.pop(0)
    node1_iface.ip_addr_add(addr=node1_addr, subnet=subnet)
    
    stdout, stderr = node1.execute(f'ip addr show {node1_iface.get_os_interface()}')
    
except Exception as e:
    print(f"Exception: {e}")

### Configure Node2

Repeat the steps to add the next available IP to the second node.

In [ ]:
try:
    node2 = slice1.get_node(name=node2_name)        
    node2_iface = node2.get_interface(network_name=network_name_1)  
    node2_addr = available_ips.pop(0)
    node2_iface.ip_addr_add(addr=node2_addr, subnet=subnet)
    
    stdout, stderr = node2.execute(f'ip addr show {node2_iface.get_os_interface()}')
    
except Exception as e:
    print(f"Exception: {e}")

## Run the ping test on Slice1

We will find the ping round trip time for this pair of sites.  Your experiment should be more interesting!


In [ ]:
try:
    node1 = slice1.get_node(name=node1_name)        

    stdout, stderr = node1.execute(f'ping -c 5 {node2_addr}')

except Exception as e:
    print(f"Exception: {e}")

## TCP capture on Slice2 
User can capture traffic on Node3 in Slice2 on the interface connected to the port-mirror service.

## Delete the Slice1

Please delete your slice when you are done with your experiment.

In [ ]:
try:
    slice1 = fablib.get_slice(name=slice_name_1)
    slice1.delete()    
except Exception as e:
    print(f"Exception: {e}")

## Delete the Slice2 (Port Mirror Slice)

Please delete your slice when you are done with your experiment.

In [ ]:
try:
    slice2 = fablib.get_slice(name=slice_name_2)
    slice2.delete()
    
except Exception as e:
    print(f"Exception: {e}")